In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
financials_df = pd.read_csv('../example_data/financials.csv')

In [4]:
financials_df['digit_count__phone_num'] = financials_df['phone_num'].str.count(r'\d')
financials_df['digit_count__cc_num'] = financials_df['cc_num'].str.count(r'\d')

In [8]:
financials_df.columns

Index(['customer_id', 'medicare_id', 'first_name', 'last_name', 'street_addr',
       'city', 'zipc', 'phone_num', 'email', 'cc_num', 'transaction_amount',
       'transaction_date', 'digit_count__phone_num', 'digit_count__cc_num'],
      dtype='object')

In [21]:
financials_df['medicare_id'].isna().mean()

0.0

In [26]:
financials_df['medicare_id'].apply(len).describe()

count    99.000000
mean     11.505051
std       0.873344
min      11.000000
25%      11.000000
50%      11.000000
75%      12.000000
max      13.000000
Name: medicare_id, dtype: float64

In [27]:
financials_df['phone_num'].apply(len).describe()

count    99.000000
mean     13.252525
std       1.886274
min       9.000000
25%      12.000000
50%      13.000000
75%      15.000000
max      17.000000
Name: phone_num, dtype: float64

In [29]:
financials_df['phone_num_clean'] =financials_df['phone_num'].apply(lambda x :x.replace(' ','')) 

In [30]:
financials_df['phone_num_clean'].apply(len).describe()

count    99.000000
mean     12.323232
std       1.671052
min       9.000000
25%      11.000000
50%      12.000000
75%      14.000000
max      16.000000
Name: phone_num_clean, dtype: float64

In [6]:
financials_df['digit_count__phone_num'].describe()

count    99.000000
mean     12.040404
std       1.477168
min       9.000000
25%      11.000000
50%      12.000000
75%      13.000000
max      16.000000
Name: digit_count__phone_num, dtype: float64

In [7]:
financials_df['digit_count__cc_num'].describe()

count    99.000000
mean     15.222222
std       1.562021
min      11.000000
25%      15.000000
50%      16.000000
75%      16.000000
max      19.000000
Name: digit_count__cc_num, dtype: float64

In [36]:
!python ../code/file_scanner.py ../example_data/financials.csv report_20_07.docx

Report generated and saved to report_20_07.docx


In [33]:
!python scanner.py financials.csv report_20_07_2024.docx

python: can't open file 'c:\\Users\\User\\Documents\\file_scanner\\notebooks\\scanner.py': [Errno 2] No such file or directory


In [33]:
import re

In [65]:

def contains_credit_card(column):
    # Regex pattern to match credit card numbers with 13 to 16 digits, 
    # allowing spaces or hyphens between digits.
    credit_card_pattern = re.compile(r'\b(?:\d[ -]*?){13,16}\b')
    
    # Apply the pattern to the column and return True if the pattern is found.
    return column.apply(lambda x: bool(credit_card_pattern.match(str(x))))

def contains_special_characters(column):
    special_char_pattern = re.compile(r'[^\w\s-]')
    return column.apply(lambda x: bool(special_char_pattern.search(str(x))))


def contains_mbi(column):
    # Regex pattern to match Medicare Beneficiary Identifier (MBI) which consists 
    # of 11 characters including letters (A-Z ,without S, L, O, I, B, and Z) and digits (0-9).
    mbi_pattern = re.compile(r'^[AC-HJ-MN-RT-Y0-9]{11}$')
    
    # Apply the pattern to the column and return True if the pattern is found.
    return column.apply(lambda x: bool(mbi_pattern.match(str(x))))

# Define Huristics Dictionary
Huristics_dict = {
                    "Credit Card Number": contains_credit_card,
                    "Special Characters": contains_special_characters,
                    "Medicare Beneficiary Identifier": contains_mbi
                }


#load data 
financials_df = pd.read_csv(file path)

#actual matching 
for title,logic in Huristics_dict.items():
    most_fit_column = financials_df.iloc[:,np.argmax(financials_df.apply(lambda x: logic(x).mean(),axis=0))].name
    pct_true =  np.max(financials_df.apply(lambda x: logic(x).mean(),axis=0))
    print(title,f": {most_fit_column}, ",f'pct match = {pct_true:.1%}')
    
## make docx report reporting the matched columns, pct match, other columns that has more than 10% matching and a time stamp 

Credit Card Number : cc_num,  pct match = 75.8%
Special Characters : email,  pct match = 100.0%
Medicare Beneficiary Identifier : medicare_id,  pct match = 74.7%


In [61]:
rule = contains_mbi

most_fit_column = financials_df.iloc[:,np.argmax(financials_df.apply(lambda x: rule(x).mean(),axis=0))].name
pct_true =  np.max(financials_df.apply(lambda x: rule(x).mean(),axis=0))

print(most_fit_column,
     f"{pct_true:.1%}")


medicare_id 74.7%


In [50]:
financials_df.iloc[:,np.argmax(financials_df.apply(lambda x: contains_mbi(x).mean(),axis=0))].name

'medicare_id'